# Assignment

Download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. Then complete 15 tasks below.

### Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
# Importing libraries
import sqlite3
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean

pd.set_option('display.max_column', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Connection to database
db = sqlite3.connect('database.sqlite')

### Task 1. Calculate the number of players with a height between 180 and 190 inclusive

In [3]:
player_data = pd.read_sql("SELECT * FROM Player;", db, parse_dates=['birthday'])

In [4]:
print(f'Shape {player_data.shape}')
player_data.head()

Shape (11060, 7)


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154


In [5]:
players_180_190 = player_data.query("180 <= height <= 190").shape[0]
players_180_190

5925

### Task 2. Calculate the number of players born in 1980

In [6]:
players_1980 = player_data.birthday.dt.year.value_counts().to_frame().query('index == 1980').values[0][0]
players_1980

434

### Task 3. Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name

In [7]:
highest_players = player_data \
    .sort_values(['weight', 'player_name'], ascending=[False, True])[:10] \
    .player_name \
    .to_list()

highest_players

['Kristof van Hout',
 'Tim Wiese',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Cassio',
 'Christopher Samba',
 'Lars Unnerstall',
 'Marcus Hahnemann',
 'Abdoulaye Faye',
 'Enoch Showunmi']

### Task 4. Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990

Structure example: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [8]:
player_data['year'] = player_data.birthday.dt.year
born_by_years = player_data.query('1980 <= year <= 1990') \
            .groupby('year', as_index=False) \
            .agg(number=('player_name', 'count'))

years_born_players = list(zip(born_by_years.year.to_list(), born_by_years.number.to_list()))
years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

### Task 5. Calculate the mean and the standard deviation of the players' height with the name `Adriano`

**Note:** Name is represented by the first part of `player_name`.

In [9]:
player_data['first_name'] = player_data.player_name.apply(lambda x: x.split(' ')[0])
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,year,first_name
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,1992,Aaron
1,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,1989,Aaron
2,3,162549,Aaron Doran,186170,1991-05-13,170.18,163,1991,Aaron
3,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198,1982,Aaron
4,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,1979,Aaron


In [10]:
adriano = player_data.query('first_name == "Adriano"').height
adriano_mean, adriano_std = adriano.mean(), adriano.std()
adriano_mean, adriano_std

(182.11800000000002, 5.361479480723788)

### Task 6. How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [11]:
day_with_min_players_born = player_data.birthday.dt.day_name().value_counts().idxmin()
day_with_min_players_born

'Sunday'

### Task 7. Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [12]:
# Let's write the query to the database to get the required data
query_1 = """
SELECT *
FROM Match
INNER JOIN League ON Match.country_id = League.country_id
"""

In [13]:
# Loading the data
matches_df = pd.read_sql(query_1, db)
matches_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,id,country_id,name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20,1,1,Belgium Jupiler League
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60,1,1,Belgium Jupiler League
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75,1,1,Belgium Jupiler League
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50,1,1,Belgium Jupiler League
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67,1,1,Belgium Jupiler League


In [14]:
league_most_matches = matches_df.groupby('name', as_index=False) \
                            .agg(matches=('match_api_id', 'count')) \
                            .sort_values(['matches', 'name'], ascending=[False, True]) \
                            .head(1) \
                            .name \
                            .to_list()[0]

league_most_matches

'England Premier League'

### Task 8. Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [15]:
# Extracting columns with players' ids
players_columns = matches_df \
                .columns[matches_df.columns.str.match(pat='(away_player_[0-9])|(home_player_[0-9])')] \
                .to_list()

# Let's have a look at the names of the columns
print(*players_columns)

home_player_1 home_player_2 home_player_3 home_player_4 home_player_5 home_player_6 home_player_7 home_player_8 home_player_9 home_player_10 home_player_11 away_player_1 away_player_2 away_player_3 away_player_4 away_player_5 away_player_6 away_player_7 away_player_8 away_player_9 away_player_10 away_player_11


In [16]:
# Defining an empty dataframe
players_matches = pd.DataFrame()

# Counting the number of plays for each player in players_columns and concatenating data to the players_matches dataframe
for column in players_columns:
    df = matches_df.pivot_table(index=column, columns='name', values='league_id', aggfunc='count', fill_value=0) \
        .sum(axis=1) \
        .to_frame() \
        .reset_index() \
        .rename(columns={column: 'player_api_id', 0: 'games_number'})
    
    players_matches = pd.concat([players_matches, df], ignore_index=True)

# Let's have a look at the final dataframe
players_matches.head()

,player_api_id,games_number
0,2984.0,19
1,5044.0,11
2,5047.0,5
3,5440.0,3
4,5710.0,27


In [17]:
# Let's group each player by his id and sum the numbers of games played
games_played = players_matches \
                .groupby('player_api_id', as_index=False) \
                .agg({'games_number': 'sum'})

# Transforming player_api_id column to int type
games_played['player_api_id'] = games_played['player_api_id'].map(int)

# Let's have a look at the final dataframe
games_played.head()

,player_api_id,games_number
0,2625,90
1,2752,81
2,2768,71
3,2770,16
4,2790,50


In [18]:
# Merging players dataframe with games_played
player_data = pd.merge(games_played, player_data, on='player_api_id')
player_data.head()

,player_api_id,games_number,id,player_name,player_fifa_api_id,birthday,height,weight,year,first_name
0,2625,90,8372,"Patryk Rachwal,18",148544,1981-01-27,175.26,154,1981,Patryk
1,2752,81,2639,Diego Mainz,135819,1982-12-29,187.96,174,1982,Diego
2,2768,71,5319,Jose Dorado,110019,1982-07-10,180.34,154,1982,Jose
3,2770,16,4339,Ignacio Gonzalez,182861,1982-05-14,180.34,161,1982,Ignacio
4,2790,50,306,Alberto Rey,110809,1974-02-15,175.26,163,1974,Alberto


In [19]:
# Finding a player with the largest number of matches played
max_matches_player = player_data.loc[player_data.games_number.idxmax()].player_name
max_matches_player

'Steve Mandanda'

### Task 9. List top-5 tuples of most correlated player's characteristics in the descending order of the absolute Pearson's coefficient value

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

In [20]:
# Loading the data
player_attributes = pd.read_sql("SELECT * FROM Player_Attributes;", db, parse_dates=['date'])
print(f'Shape {player_attributes.shape}')
player_attributes.head()

Shape (183978, 42)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [21]:
# Let's drop unrelevant columns
attributes = player_attributes \
                .drop(columns=['id', 'player_fifa_api_id', 'player_api_id',
                               'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])

attributes.head()

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,62.0,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [22]:
# Getting the matrix with the Pearson's correlation
corr_matrix = attributes.corr().abs()

In [23]:
# Let's slise the matrix by its axis and have a look at the result
mtx = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
mtx.style \
    .background_gradient(cmap='viridis_r') \
    .highlight_null('white') \
    .format("{:.4}", na_rep="")

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
overall_rating,,0.7668,0.3577,0.3293,0.3141,0.4584,0.3615,0.3543,0.3578,0.3496,0.435,0.4443,0.2457,0.2548,0.2399,0.7692,0.1604,0.428,0.259,0.3275,0.3187,0.3924,0.3239,0.2504,0.37,0.4311,0.3932,0.1334,0.1653,0.1289,0.02798,0.00441,0.02568,0.005709,0.005687
potential,,,0.2778,0.2867,0.2077,0.3826,0.3013,0.3387,0.2961,0.2634,0.3437,0.4014,0.338,0.3401,0.293,0.5802,0.202,0.3257,0.1748,0.2605,0.1269,0.3128,0.1648,0.1657,0.3278,0.3789,0.3164,0.05576,0.08382,0.06398,0.01125,0.004742,0.0892,0.003406,0.003962
crossing,,,,0.5769,0.3697,0.7901,0.6379,0.8091,0.7891,0.7076,0.682,0.8073,0.599,0.5794,0.5992,0.3845,0.5185,0.6563,0.02132,0.5644,0.07029,0.7159,0.3236,0.306,0.6821,0.6932,0.5715,0.2352,0.2853,0.2755,0.6039,0.5956,0.3496,0.5976,0.6016
finishing,,,,,0.3735,0.5794,0.8513,0.7847,0.6912,0.6322,0.3376,0.7197,0.5289,0.5095,0.5542,0.3545,0.394,0.727,0.008953,0.3467,0.05305,0.806,0.04371,0.1494,0.7984,0.6515,0.7204,0.2846,0.2297,0.2609,0.4792,0.4651,0.2877,0.4707,0.4732
heading_accuracy,,,,,,0.5487,0.3916,0.4005,0.3211,0.3076,0.3586,0.551,0.2001,0.2673,0.06922,0.2953,0.07895,0.5409,0.2868,0.4775,0.4931,0.4068,0.5757,0.4528,0.4116,0.3372,0.4313,0.4615,0.4806,0.4418,0.6654,0.6479,0.3915,0.6477,0.6514
short_passing,,,,,,,0.6403,0.788,0.7323,0.6925,0.7975,0.89,0.5033,0.4914,0.5106,0.4593,0.4617,0.7218,0.06015,0.61,0.09208,0.7288,0.4538,0.4243,0.6776,0.7653,0.6095,0.3502,0.4157,0.381,0.6933,0.6895,0.4133,0.6907,0.6929
volleys,,,,,,,,0.7844,0.7522,0.6828,0.4141,0.7496,0.5133,0.4941,0.5598,0.3972,0.4158,0.7465,0.02323,0.3826,0.03611,0.8144,0.1279,0.0376,0.7783,0.6899,0.7118,0.169,0.1069,0.1264,0.5088,0.487,0.279,0.491,0.493
dribbling,,,,,,,,,0.8104,0.7059,0.5743,0.9008,0.6982,0.6695,0.7031,0.3777,0.5461,0.7441,0.008559,0.5254,0.1116,0.8063,0.2028,0.1067,0.7933,0.7327,0.6577,0.004128,0.06696,0.04604,0.6527,0.6505,0.4271,0.6534,0.656
curve,,,,,,,,,,0.7974,0.5857,0.7987,0.5494,0.5169,0.6193,0.3929,0.4931,0.6947,0.01696,0.4543,0.1144,0.7835,0.2036,0.1363,0.7202,0.7269,0.6485,0.03368,0.09521,0.08121,0.5572,0.5458,0.3335,0.5507,0.5523
free_kick_accuracy,,,,,,,,,,,0.5991,0.7197,0.4308,0.3948,0.505,0.3681,0.4304,0.6828,0.03326,0.4161,0.05618,0.7727,0.2324,0.178,0.6542,0.6974,0.6658,0.07451,0.1345,0.107,0.4988,0.4921,0.2711,0.4948,0.4963


In [24]:
# Looping each row and column and getting tuples with the correlation above 0.87
corr_tuples = []
for column in mtx.columns[:-1]:
    for row in mtx.columns[1:]:
        corr = mtx[mtx.index == column][row][0]
        if corr > 0.87:
            corr_tuples.append((column, row, corr))

# Getting the final dataframe
high_corr = pd.DataFrame(corr_tuples, columns=['col_name', 'row_name', 'corr']).sort_values('corr', ascending=False)[:5]
high_corr

,col_name,row_name,corr
11,gk_positioning,gk_reflexes,0.966711
10,gk_handling,gk_reflexes,0.966326
9,gk_handling,gk_positioning,0.966301
5,standing_tackle,sliding_tackle,0.953104
3,marking,standing_tackle,0.950134


In [25]:
# Listing top-5 tuples of most correlated player's characteristic
top_correlated_features = list(zip(high_corr.row_name.to_list(), high_corr.col_name.to_list()))
top_correlated_features

[('gk_reflexes', 'gk_positioning'),
 ('gk_reflexes', 'gk_handling'),
 ('gk_positioning', 'gk_handling'),
 ('sliding_tackle', 'standing_tackle'),
 ('standing_tackle', 'marking')]

### Task 10. Find top-5 most similar players to Neymar whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same

**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`).<br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization.

In [26]:
# Let's drop unrelevant columns from the player_data
players_names = player_data.drop(columns=['games_number', 'id', 'player_fifa_api_id', 'birthday', 'height', 'weight', 'year'])

# Merging two dataframes
df = pd.merge(player_attributes, players_names, on='player_api_id')

# Let's drop unrelevant columns and rows with NA values
df = df.drop(columns=['id', 'player_fifa_api_id', 'player_api_id', 'preferred_foot',
                      'attacking_work_rate', 'defensive_work_rate', 'first_name']).dropna()

# Let's have a look at the final dataframe
df.head()

,date,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,player_name
0,2016-02-18,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,Aaron Appindangoye
1,2015-11-19,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,Aaron Appindangoye
2,2015-09-21,62.0,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0,Aaron Appindangoye
3,2015-03-20,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,Aaron Appindangoye
4,2007-02-22,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,Aaron Appindangoye


In [27]:
# Getting the freshest data for each player
freshest_data = df.sort_values(['player_name', 'date']).drop_duplicates('player_name', keep='last')
freshest_data.head()

,date,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,player_name
0,2016-02-18,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,Aaron Appindangoye
5,2016-04-21,74.0,76.0,80.0,53.0,58.0,71.0,40.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,62.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,Aaron Cresswell
38,2016-01-07,65.0,67.0,64.0,58.0,60.0,66.0,52.0,66.0,67.0,63.0,62.0,67.0,77.0,74.0,85.0,51.0,92.0,61.0,65.0,66.0,71.0,57.0,60.0,55.0,64.0,68.0,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0,Aaron Doran
64,2016-04-21,69.0,69.0,57.0,21.0,68.0,65.0,48.0,56.0,38.0,40.0,61.0,62.0,33.0,34.0,62.0,57.0,41.0,59.0,71.0,49.0,90.0,29.0,70.0,62.0,26.0,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0,Aaron Galindo
87,2015-12-24,70.0,70.0,46.0,34.0,73.0,65.0,33.0,47.0,42.0,25.0,58.0,58.0,33.0,31.0,41.0,68.0,52.0,48.0,67.0,60.0,75.0,26.0,66.0,68.0,29.0,41.0,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0,Aaron Hughes


In [28]:
# Getting Neymar's characteristics
neymar = freshest_data.query('player_name == "Neymar"').drop(columns=['date', 'player_name'])
neymar_index = neymar.index[0]
neymar_values = neymar.values
print(neymar_index)
print(*neymar_values)

131464
[90. 94. 72. 88. 62. 78. 83. 94. 78. 79. 74. 93. 91. 90. 92. 86. 84. 78.
 61. 79. 45. 73. 56. 36. 89. 79. 81. 21. 24. 33.  9.  9. 15. 15. 11.]


In [29]:
# Getting other players' characteristics
other_players = freshest_data.drop(neymar_index, axis=0)
other_players = other_players.drop(columns='date').set_index('player_name')
other_players[:5]

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
player_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aaron Appindangoye,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
Aaron Cresswell,74.0,76.0,80.0,53.0,58.0,71.0,40.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,62.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
Aaron Doran,65.0,67.0,64.0,58.0,60.0,66.0,52.0,66.0,67.0,63.0,62.0,67.0,77.0,74.0,85.0,51.0,92.0,61.0,65.0,66.0,71.0,57.0,60.0,55.0,64.0,68.0,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0
Aaron Galindo,69.0,69.0,57.0,21.0,68.0,65.0,48.0,56.0,38.0,40.0,61.0,62.0,33.0,34.0,62.0,57.0,41.0,59.0,71.0,49.0,90.0,29.0,70.0,62.0,26.0,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0
Aaron Hughes,70.0,70.0,46.0,34.0,73.0,65.0,33.0,47.0,42.0,25.0,58.0,58.0,33.0,31.0,41.0,68.0,52.0,48.0,67.0,60.0,75.0,26.0,66.0,68.0,29.0,41.0,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0


In [30]:
# Measuring Euclidean distance
other_players['distance'] = other_players \
    .apply(lambda row: euclidean(neymar_values, [row[i] for i in range(len(other_players.columns))]), axis=1)

# Getting the final dataframe
other_players.reset_index(inplace=True)
other_players = other_players.sort_values('distance')
other_players[:5]

,player_name,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,distance
7920,Paulo Dybala,81.0,87.0,72.0,87.0,60.0,78.0,80.0,89.0,81.0,82.0,58.0,90.0,90.0,85.0,91.0,82.0,85.0,77.0,69.0,79.0,55.0,83.0,48.0,31.0,85.0,79.0,76.0,15.0,21.0,21.0,9.0,11.0,9.0,10.0,11.0,33.837849
2658,Eden Hazard,88.0,90.0,80.0,81.0,57.0,85.0,79.0,94.0,82.0,79.0,82.0,90.0,93.0,87.0,92.0,86.0,90.0,79.0,59.0,74.0,63.0,82.0,54.0,41.0,84.0,86.0,86.0,25.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,34.957117
3630,Giovani dos Santos,79.0,80.0,74.0,81.0,61.0,73.0,79.0,85.0,73.0,80.0,71.0,85.0,87.0,86.0,89.0,80.0,89.0,80.0,65.0,68.0,45.0,79.0,53.0,28.0,80.0,70.0,70.0,21.0,28.0,23.0,6.0,9.0,11.0,7.0,5.0,38.314488
900,Arjen Robben,89.0,89.0,80.0,85.0,51.0,85.0,86.0,93.0,86.0,83.0,73.0,90.0,90.0,90.0,91.0,91.0,91.0,86.0,61.0,73.0,65.0,90.0,47.0,39.0,88.0,84.0,80.0,29.0,26.0,26.0,10.0,8.0,11.0,5.0,15.0,40.024992
5802,Lionel Messi,94.0,94.0,80.0,93.0,71.0,88.0,85.0,96.0,89.0,90.0,79.0,96.0,95.0,90.0,92.0,92.0,95.0,80.0,68.0,75.0,59.0,88.0,48.0,22.0,90.0,90.0,74.0,13.0,23.0,21.0,6.0,11.0,15.0,14.0,8.0,43.543082


In [31]:
# Saving the result to a list
neymar_similarities = other_players.player_name[:5].to_list()
neymar_similarities

['Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

### Task 11. Calculate the number of home matches played by the Borussia Dortmund team in Germany 1. Bundesliga in season 2008/2009

In [32]:
# Let's write the query to the database to get the required data
query_2 = """
SELECT
    league_name,
    team_long_name,
    season,
    home_team_api_id,
    away_team_api_id
FROM (
        SELECT
            season,
            home_team_api_id,
            away_team_api_id,
            name AS league_name
        FROM match
        INNER JOIN league ON match.country_id = league.country_id
    ) AS t
INNER JOIN Team ON t.home_team_api_id = Team.team_api_id
"""

In [33]:
# Loading the data
plays_df = pd.read_sql(query_2, db, parse_dates=['date'])
print(f'Shape {plays_df.shape}')
plays_df.head()

Shape (25979, 5)


,league_name,team_long_name,season,home_team_api_id,away_team_api_id
0,Belgium Jupiler League,KRC Genk,2008/2009,9987,9993
1,Belgium Jupiler League,SV Zulte-Waregem,2008/2009,10000,9994
2,Belgium Jupiler League,KSV Cercle Brugge,2008/2009,9984,8635
3,Belgium Jupiler League,KAA Gent,2008/2009,9991,9998
4,Belgium Jupiler League,FCV Dender EH,2008/2009,7947,9985


In [34]:
borussia_bundesliga_2008_2009_matches = plays_df \
    .query('season == "2008/2009" & league_name == "Germany 1. Bundesliga" & team_long_name == "Borussia Dortmund"') \
    .shape[0]

borussia_bundesliga_2008_2009_matches

17

### Task 12. Find a team having the most matches (both home and away!) in the Germany 1. Bundesliga in 2008/2009 season. Return number of matches

In [35]:
team_most_matches_bundesliga_2008_2009 = plays_df \
    .query('season == "2008/2009" & league_name == "Germany 1. Bundesliga"') \
    .groupby('team_long_name', as_index=False) \
    .agg(home_plays=('home_team_api_id', 'count'),
         away_plays=('away_team_api_id', 'count')) \
    .sum(axis=1)[0]

team_most_matches_bundesliga_2008_2009

34

### Task 13. Count total number of Arsenal matches (both home and away!) in the 2015/2016 season which they have won

**Note:** Winning a game means scoring **more** goals than an opponent.

In [36]:
# Let's write the query to the database to get the required data
query_3 = """
SELECT
    date,
    league_name,
    season,
    home_team,
    team_long_name AS away_team,
    home_team_goal,
    away_team_goal,
    CASE WHEN (home_team_goal - away_team_goal) > 0 THEN 'won'
         WHEN (home_team_goal - away_team_goal) = 0 THEN 'draw'
         ELSE 'lost' END as home_result,
    CASE WHEN (away_team_goal - home_team_goal) > 0 THEN 'won'
         WHEN (away_team_goal - home_team_goal) = 0 THEN 'draw'
         ELSE 'lost' END as away_result
FROM (    
        SELECT
            date,
            season,
            league_name,
            team_long_name AS home_team,
            away_team_api_id,
            home_team_goal,
            away_team_goal
        FROM (
                SELECT
                    season,
                    date,
                    home_team_api_id,
                    away_team_api_id,
                    home_team_goal,
                    away_team_goal,
                    name AS league_name
                FROM match
                INNER JOIN league ON match.country_id = league.country_id
             ) AS t
        INNER JOIN Team ON t.home_team_api_id = Team.team_api_id
      ) AS b
INNER JOIN Team ON b.away_team_api_id = Team.team_api_id
"""

In [37]:
# Loading the data
plays_data = pd.read_sql(query_3, db, parse_dates=['date'])
plays_data.head()

,date,league_name,season,home_team,away_team,home_team_goal,away_team_goal,home_result,away_result
0,2008-08-17,Belgium Jupiler League,2008/2009,KRC Genk,Beerschot AC,1,1,draw,draw
1,2008-08-16,Belgium Jupiler League,2008/2009,SV Zulte-Waregem,Sporting Lokeren,0,0,draw,draw
2,2008-08-16,Belgium Jupiler League,2008/2009,KSV Cercle Brugge,RSC Anderlecht,0,3,lost,won
3,2008-08-17,Belgium Jupiler League,2008/2009,KAA Gent,RAEC Mons,5,0,won,lost
4,2008-08-16,Belgium Jupiler League,2008/2009,FCV Dender EH,Standard de Liège,1,3,lost,won


In [38]:
# Counting the number of matches won at home by Arsenal in season 2015/2016
arsenal_home = plays_data.query('home_team == "Arsenal" & season == "2015/2016"') \
    .groupby('home_result', as_index=False) \
    .agg(results=('away_result', 'count')) \
    .query('home_result == "won"') \
    .results \
    .values[0]

In [39]:
# Counting the number of matches won played away by Arsenal in season 2015/2016
arsenal_away = plays_data.query('away_team == "Arsenal" & season == "2015/2016"') \
    .groupby('away_result', as_index=False) \
    .agg(results=('away_result', 'count')) \
    .query('away_result == "won"') \
    .results \
    .values[0]

In [40]:
# Getting the total number
arsenal_won_matches_2015_2016 = arsenal_home + arsenal_away
arsenal_won_matches_2015_2016

20

### Task 14. Find a team with the highest win rate in the 2015/2016 season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [41]:
# Getting the results of played matches at home for each team
home_results = plays_data.query('season == "2015/2016"') \
    .pivot_table(index='home_team', columns='home_result', values='home_team_goal', aggfunc='count', fill_value=0) \
    .reset_index() \
    .rename(columns={'home_team': 'team'})

home_results.head()

home_result,team,draw,lost,won
0,1. FC Köln,5,7,5
1,1. FSV Mainz 05,4,5,8
2,ADO Den Haag,8,5,4
3,AS Monaco,6,3,10
4,AS Saint-Étienne,4,5,10


In [42]:
# Getting the results of played away matches for each team
away_results = plays_data.query('season == "2015/2016"') \
    .pivot_table(index='away_team', columns='away_result', values='away_team_goal', aggfunc='count', fill_value=0) \
    .reset_index() \
    .rename(columns={'away_team': 'team'})

away_results.head()

away_result,team,draw,lost,won
0,1. FC Köln,8,4,5
1,1. FSV Mainz 05,4,7,6
2,ADO Den Haag,5,6,6
3,AS Monaco,8,4,7
4,AS Saint-Étienne,3,9,7


In [43]:
# Merging two dataframes and measuring win_rate for each team
full_table = pd.merge(home_results, away_results, on='team', suffixes=('_home', '_away'))
full_table['win_rate'] = (full_table.won_home + full_table.won_away) / full_table.sum(axis=1)
full_table = full_table.sort_values('win_rate', ascending=False)
full_table.head()

,team,draw_home,lost_home,won_home,draw_away,lost_away,won_away,win_rate
144,SL Benfica,0,2,15,1,2,14,0.852941
46,FC Bayern Munich,1,1,15,3,1,13,0.823529
154,Sporting CP,3,1,13,2,1,14,0.794118
118,Paris Saint-Germain,3,1,15,3,1,15,0.789474
116,PSV,3,1,13,3,1,13,0.764706


In [44]:
# Getting the name of the team
team_highest_winrate_2015_2016 = full_table.loc[full_table.win_rate.idxmax()].team
team_highest_winrate_2015_2016

'SL Benfica'

### Task 15. Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season. Return number of days in that gap

**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [45]:
plays_data.head()

,date,league_name,season,home_team,away_team,home_team_goal,away_team_goal,home_result,away_result
0,2008-08-17,Belgium Jupiler League,2008/2009,KRC Genk,Beerschot AC,1,1,draw,draw
1,2008-08-16,Belgium Jupiler League,2008/2009,SV Zulte-Waregem,Sporting Lokeren,0,0,draw,draw
2,2008-08-16,Belgium Jupiler League,2008/2009,KSV Cercle Brugge,RSC Anderlecht,0,3,lost,won
3,2008-08-17,Belgium Jupiler League,2008/2009,KAA Gent,RAEC Mons,5,0,won,lost
4,2008-08-16,Belgium Jupiler League,2008/2009,FCV Dender EH,Standard de Liège,1,3,lost,won


In [46]:
# Getting dates of matches played at home for each team
home_team = plays_data.query('league_name == "England Premier League" & season == "2010/2011"') \
    .groupby(['home_team', 'date'], as_index=False) \
    .agg({'home_result': 'count'})

home_team = home_team.drop(columns=['home_result'])
home_team.rename(columns={'home_team': 'team'}, inplace=True)

home_team.head()

,team,date
0,Arsenal,2010-08-21
1,Arsenal,2010-09-11
2,Arsenal,2010-09-25
3,Arsenal,2010-10-16
4,Arsenal,2010-10-30


In [47]:
# Getting dates of matches played away for each team
away_team = plays_data.query('league_name == "England Premier League" & season == "2010/2011"') \
    .groupby(['away_team', 'date'], as_index=False) \
    .agg({'home_result': 'count'})

away_team = away_team.drop(columns=['home_result'])
away_team.rename(columns={'away_team': 'team'}, inplace=True)

away_team.head()

,team,date
0,Arsenal,2010-08-15
1,Arsenal,2010-08-28
2,Arsenal,2010-09-18
3,Arsenal,2010-10-03
4,Arsenal,2010-10-24


In [48]:
# Concatenating two dataframes
games_df = pd.concat([home_team, away_team], ignore_index=True)

# Sorting values by the name of a team and date
games_df = games_df.sort_values(['team', 'date'], ascending=[True, False])

# Setting team names as index
games_df.set_index(['team'], inplace=True)

# Defining a new column
games_df['time_gap'] = pd.Timedelta(0, 'd')

# Getting difference between days of matches for each team
for idx in games_df.index:
    games_df.time_gap[idx] = games_df.date[idx].diff(1).abs()

# Getting the final dataframe
games_df.reset_index(inplace=True)
games_df.head()

,team,date,time_gap
0,Arsenal,2011-05-22,NaT
1,Arsenal,2011-05-15,7 days
2,Arsenal,2011-05-08,7 days
3,Arsenal,2011-05-01,7 days
4,Arsenal,2011-04-24,7 days


In [49]:
# Let's have a look at the team with the maximum days' gap between matches
games_df.loc[games_df.time_gap.idxmax()]

team            Birmingham City
date        2011-02-15 00:00:00
time_gap       18 days 00:00:00
Name: 88, dtype: object

In [50]:
# Getting the number of days
highest_gap_england_2010_2011 = games_df.loc[games_df.time_gap.idxmax()].time_gap.days
highest_gap_england_2010_2011

18